# ПРОГНОЗИРОВАНИЕ №8.2 - ABC XYZ

In [1]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgrmnm79b
  JVM stdout: /tmp/tmpgrmnm79b/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpgrmnm79b/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.2
H2O_cluster_version_age:,2 months and 3 days
H2O_cluster_name:,H2O_from_python_unknownUser_hos2y4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.926 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
#file = '/content/drive/MyDrive/Colab Notebooks/Diplom/df4eda2clean2.csv'
file = 'df2prep4regressionV8.csv'
df = h2o.import_file(file)
df = df.drop('C1')
df = df.drop('Revenue_cat')
df = df.drop('Revenue_T')
df

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Zip,Units,Manufacturer,MonthNo,Model_id_number,CategoryInSegment,abc_xyz_class
6111,12,Natura,12,44,UrbanConvenience,BY
92879,7,Pirum,12,32,RuralProductivity,BY
92879,7,Pirum,12,31,RuralProductivity,BY
32901,3,VanArsdel,12,94,UrbanModeration,AY
84770,3,Currus,12,8,RuralSelect,AZ
95380,3,Abbas,12,33,UrbanModeration,BZ
14559,2,Natura,12,14,YouthYouth,BY
16301,2,Natura,12,14,YouthYouth,BY
48439,2,Quibus,12,4,RuralSelect,BX
21787,2,VanArsdel,12,70,UrbanModeration,AY


In [3]:
train, valid, unseen = df.split_frame(ratios=[0.7, 0.15])

In [4]:
train

Zip,Units,Manufacturer,MonthNo,Model_id_number,CategoryInSegment,abc_xyz_class
6111,12,Natura,12,44,UrbanConvenience,BY
92879,7,Pirum,12,32,RuralProductivity,BY
92879,7,Pirum,12,31,RuralProductivity,BY
32901,3,VanArsdel,12,94,UrbanModeration,AY
84770,3,Currus,12,8,RuralSelect,AZ
95380,3,Abbas,12,33,UrbanModeration,BZ
14559,2,Natura,12,14,YouthYouth,BY
16301,2,Natura,12,14,YouthYouth,BY
48439,2,Quibus,12,4,RuralSelect,BX
21787,2,VanArsdel,12,70,UrbanModeration,AY


In [5]:
valid

Zip,Units,Manufacturer,MonthNo,Model_id_number,CategoryInSegment,abc_xyz_class
75287,1,VanArsdel,12,54,UrbanModeration,AX
78947,1,VanArsdel,12,54,UrbanModeration,AX
6093,1,VanArsdel,12,54,UrbanModeration,AX
89117,1,Currus,12,5,UrbanExtreme,AY
11931,1,VanArsdel,12,11,UrbanModeration,AX
11757,1,Currus,12,24,UrbanExtreme,AY
11435,1,Currus,12,24,UrbanExtreme,AY
10924,1,Currus,12,24,UrbanExtreme,AY
11208,1,Currus,12,24,UrbanExtreme,AY
85215,1,Currus,12,24,UrbanExtreme,AY


In [6]:
unseen

Zip,Units,Manufacturer,MonthNo,Model_id_number,CategoryInSegment,abc_xyz_class
70578,1,VanArsdel,12,69,UrbanConvenience,AX
85629,1,Natura,12,36,UrbanExtreme,AY
81506,1,VanArsdel,12,54,UrbanModeration,AX
30534,1,VanArsdel,12,54,UrbanModeration,AX
51041,1,VanArsdel,12,11,UrbanModeration,AX
15423,1,VanArsdel,12,11,UrbanModeration,AX
50211,1,VanArsdel,12,11,UrbanModeration,AX
77469,1,VanArsdel,12,11,UrbanModeration,AX
68123,1,VanArsdel,12,11,UrbanModeration,AX
12074,1,VanArsdel,12,11,UrbanModeration,AX


In [7]:
y = "abc_xyz_class"
x=train.columns
x.remove(y)

# Тренировка модели

nfolds=0 - это отключает перексрестную проверку и точность будет проверятся на указанной валидационной выборке

In [8]:
aml = H2OAutoML(
    max_models = 10,
    seed = 1,
    nfolds = 0 # отключение кросс валидации
)
aml.train(x = x, y = y, training_frame = train, validation_frame = valid)

AutoML progress: |██████████████████████████████████
16:30:51.313: GBM_1_AutoML_1_20230928_160209 [GBM def_5] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_1_AutoML_1_20230928_160209.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (166.6 KB per tree x 10000 > 1.58 GB) - try decreasing ntrees and/or max_depth or increasing min_rows!


█████████████████████████████
17:28:30.96: GBM_4_AutoML_1_20230928_160209 [GBM def_4] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_4_AutoML_1_20230928_160209.  Details: ERRR on field: _ntrees: The tree model will not fit in the driver node's memory (71.6 KB per tree x 10000 > 660.4 MB) - try decreasing ntrees and/or max_depth or increasing min_rows!


 (cancelled) 100%


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_9a73bbe9a1f577a83ff1a62cfb3f4fda> was cancelled by the user.

In [ ]:
lb = aml.leaderboard
lb

In [ ]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
model_ids

In [ ]:
# Get the "All Models" Stacked Ensemble model
#se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
se = h2o.get_model([mid for mid in model_ids if "AutoML" in mid][0])
se

In [ ]:
# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
metalearner.coef_norm()

In [ ]:
%matplotlib inline
metalearner.std_coef_plot()

# Что с важностью признаков

In [ ]:
for model_id in model_ids:
  try:
    print(f"Model ID: {model_id}")
    aml_unit = h2o.get_model(model_id)
    aml_unit.varimp_plot()
  except:
    print(f"{model_id} - ERROR PLOTTING") 

# Что с переобученносью

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score

for model_id in model_ids:
  try:
    print(f"Model ID: {model_id}")
    aml_unit = h2o.get_model(model_id)
    perf = aml_unit.model_performance(valid)
    print(perf)
    print(f"R2 cross validation: {aml_unit.r2()}")
    print(f"R2 test validation: {aml_unit.r2(valid=True)}")
    # прогноз лидирующей моделю на валидационной выборке
    val_pred=aml_unit.predict(unseen).as_data_frame()
    # из валидационного датасеат возбму столбец Stay и сконвертиую его в пандас датафрейм
    val_test = unseen['abc_xyz_class'].as_data_frame()
    r2_unseen = r2_score(val_test, val_pred)
    print(f"R2 unseen dataset validation: {r2_unseen}")
    print("-------------------\n-------------------")
  except:
    print(f"{model_id} - ERROR CALCULATE PERFOMANCE")

# Графии кривых обучения

In [ ]:
for model_id in model_ids:
  try:
    learning_curve_plot = h2o.get_model(model_id).learning_curve_plot()
  except:
    print(model_id, "- ERROR PLOT")

# Сохранение лучше модели

Довольно стаблильный показатель по Р-квадарт

In [ ]:
#besy_model_id = 'StackedEnsemble_BestOfFamily_1_AutoML_4_20230921_114817'

In [ ]:
# Сохранение лучшей модели
#besy_model = h2o.get_model(besy_model_id)
#h2o.save_model(besy_model, path = "./models")